# Aprendizado de Máquina - Exemplo do K-NN

#### 1 - Importar os módulos necessários e carregar a base de dados de frutas

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
fruits = pd.read_table('./CSV/fruit_data_with_colors.txt')
fruits.head()

#### 2 - Mapeamento para facilitar a interpretação dos resultados

In [ ]:
# criar um mapeamento do valor do rótulo da fruta para o nome da fruta para facilitar a interpretação dos resultados
lookup_fruit_name = dict(zip(fruits.fruit_label.unique(), fruits.fruit_name.unique() ))
lookup_fruit_name

#### 3 - Separando conjuto de treinamento de conjunto de testes

In [ ]:
X = fruits[['height', 'width', 'mass', 'color_score']]
y = fruits['fruit_label']
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0)

#### 4 - Examinando inicialmente os dados com uma matriz de dispersão

In [ ]:
# plotando uma matriz de dispersão ou gráfico de pares de características
from matplotlib import cm

cmap = cm.get_cmap('gnuplot')
scatter = pd.plotting.scatter_matrix(X_train, c=y_train, marker='o', s=40, hist_kwds={'bins':15}, figsize=(9,9), cmap=cmap )



#### 5 - Gráfico de dispersão 3D

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')
ax.scatter(X_train['width'], X_train['height'], X_train['color_score'], c = y_train, marker = 'o', s=100)
ax.set_xlabel('width')
ax.set_ylabel('height')
ax.set_zlabel('color_score')
plt.show()

####  6 - Separar (novamente) o conjunto de treinamento do conjunto de testes

In [ ]:
X = fruits[['height', 'width', 'mass']]
y = fruits['fruit_label']
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0)

# 75% / 25% padrão


#### 7 - Cria o objeto Classificador

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 5)


#### 8 - Treina o classificador usando o conjunto de dados de treinamento

In [ ]:
knn.fit(X_train, y_train)
len(X_train)
len(X_test)


#### 9 - Estimar a precisão do classificador em dados futuros, usando os dados de teste

In [ ]:
knn.score(X_test, y_test)

#### 10 - Usar o classificador k-NN já treinado para classificar objetos inéditos

In [ ]:
# fruta com massa 20g, largura 4.3 cm e altura 5.5 cm
fruit_prediction = knn.predict([[20, 4.3, 5.5], [100, 6.3, 8.5]])
print(fruit_prediction)
#lookup_fruit_name[fruit_prediction]

In [ ]:
# fruta com massa 100g, largura 6.3 cm e altura 8.5 cm
knn.predict?

#### 11 - Plotar os limites de decisão do classificador k-NN

In [ ]:
from sklearn import neighbors
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.patches as mpatches
import graphviz
from sklearn.tree import export_graphviz
import matplotlib.patches as mpatches

def plot_fruit_knn(X, y, n_neighbors, weights):
    X_mat = X[['height', 'width']].as_matrix()
    y_mat = y.as_matrix()

    # Mapas de cor
    cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF','#AFAFAF'])
    cmap_bold  = ListedColormap(['#FF0000', '#00FF00', '#0000FF','#AFAFAF'])

    clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
    clf.fit(X_mat, y_mat)

    # Atribuir uma cor no mapa de cores a cada ponto da malha
    
    mesh_step_size = .01
    plot_symbol_size = 50
    
    x_min, x_max = X_mat[:, 0].min() - 1, X_mat[:, 0].max() + 1
    y_min, y_max = X_mat[:, 1].min() - 1, X_mat[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, mesh_step_size),
                         np.arange(y_min, y_max, mesh_step_size))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    Z = Z.reshape(xx.shape)
    plt.figure()
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

    # Plotar os pontos de treinamento
    plt.scatter(X_mat[:, 0], X_mat[:, 1], s=plot_symbol_size, c=y, cmap=cmap_bold, edgecolor = 'black')
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())

    patch0 = mpatches.Patch(color='#FF0000', label='apple')
    patch1 = mpatches.Patch(color='#00FF00', label='mandarin')
    patch2 = mpatches.Patch(color='#0000FF', label='orange')
    patch3 = mpatches.Patch(color='#AFAFAF', label='lemon')
    plt.legend(handles=[patch0, patch1, patch2, patch3])

        
    plt.xlabel('height (cm)')
    plt.ylabel('width (cm)')
    
    plt.show()


plot_fruit_knn(X_train, y_train, 5, 'uniform')   # 5 vizinhos

#### 12 - Quão sensível é a precisão da classificação k-NN para a escolha do parâmetro 'k'?

In [ ]:
k_range = range(1,20)
scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train, y_train)
    scores.append(knn.score(X_test, y_test))

plt.figure()
plt.xlabel('k')
plt.ylabel('accuracy')
plt.scatter(k_range, scores)
plt.xticks([0,5,10,15,20]);